In [ ]:
!nvidia-smi

Wed Aug 16 18:35:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!chmod +x setup.sh
!./setup.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00

In [ ]:
project_name = 'LLM-for-code-intelligence-Project'
import os

path= f'/content/drive/MyDrive/{project_name}'

if not os.path.exists(path):
  os.mkdir(path)

os.chdir(path)

repo_name = 'LLM-for-code-intelligence'
repo_path = f'{path}/{repo_name}'
url = f'https://github.com/ammarnasr/{repo_name}.git'



if not os.path.exists(repo_path):
    #clone the repo
    print('Cloning the repo...')
    !git clone $url
else:
    #pull the repo
    print('Pulling the repo...')
    !git -C $repo_name pull

os.chdir(repo_path)

print(f'Current Dir: {os.getcwd()}')

Pulling the repo...
Already up to date.
Current Dir: /content/drive/MyDrive/LLM-for-code-intelligence-Project/LLM-for-code-intelligence


#### Init

In [ ]:
import os
import json
import torch
import pickle
from tqdm.auto import tqdm
from datasets import load_dataset
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelWithLMHead
os.chdir('/content/drive/MyDrive/LLM-for-code-intelligence-Project/LLM-for-code-intelligence/Evaluation/Perplexity')


global DEVICE
global MODEL
global TOKENIZER
global MAX_LENGTH
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_perplexity(text, stride):
    encodings = TOKENIZER(text, return_tensors="pt").to(DEVICE)
    seq_len = encodings.input_ids.size(1)
    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + MAX_LENGTH, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(DEVICE)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = MODEL(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
        nlls.append(neg_log_likelihood)
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl


def save_github_code_eval_subset(language, license, size=1000, shuffle=True):
    """
    Saves a subset of the github code dataset to be used for evaluation.
    :param language: Programming language of the code
    :param license: License of the code
    :param size: Size of the subset
    :param shuffle: Whether to shuffle the dataset before taking the subset
    :return: None
    """
    print(f"Getting codepaarot/github-code dataset for {language} with {license} license and {size} samples from HuggingFace...")
    ds = load_dataset("codeparrot/github-code", languages=[language], licenses=[license], streaming=True, split="train")
    if shuffle:
        ds = ds.shuffle(buffer_size=size)
    ds = ds.take(size)
    evaluation_dataset = []

    for item in tqdm(ds, total=size, desc="Saving evaluation data Locally"):
        evaluation_dataset.append(item['code'])

    eval_data_name = f"./data/evaluation_data_{language}_{license}_{size}.pkl"

    with open(eval_data_name, 'wb') as f:
        pickle.dump(evaluation_dataset, f)


def load_saved_github_code_eval_subset(language, license, size=1000):
    """
    Loads a subset of the github code dataset to be used for evaluation.
    :param language: Programming language of the code
    :param license: License of the code
    :param size: Size of the subset
    :return: None
    """
    eval_data_name = f"./data/evaluation_data_{language}_{license}_{size}.pkl"
    if not os.path.exists(eval_data_name):
        print(f"Saved evaluation data NOT found. Saving now...")
        save_github_code_eval_subset(language, license, size)
    else:
        print(f"FOUND saved evaluation data")
    with open(eval_data_name, 'rb') as f:
        evaluation_dataset = pickle.load(f)
    return evaluation_dataset


def save_results_dict(results_dict, language, license, stride, res_dir):
    """
    Saves a results dictionary to be used for evaluation to a json file.
    :param results_dict: Dictionary containing the results
    :param language: Programming language of the code
    :param license: License of the code
    :param size: Size of the subset
    :param res_dir: Directory to save the results
    :return: None
    """
    #check if directory name results exists
    if not os.path.exists('results'):
        os.makedirs('results')
    #check if res_dir exists in results
    final_res_dir = f"./results/{res_dir}"
    if not os.path.exists(final_res_dir):
        os.makedirs(final_res_dir)
    results_name = f"{final_res_dir}/results_{language}_{license}_{stride}.json"
    results_name_short = f"{final_res_dir}/results_{language}_{license}_{stride}_short.json"

    with open(results_name, 'w') as f:
        json.dump(results_dict, f)
    print(f"Saved results to {results_name}")

    # Delete text to save space
    del results_dict['text']
    with open(results_name_short, 'w') as f:
        json.dump(results_dict, f)
    print(f"Saved short results to {results_name_short}")


def main(languages, licenses, stride, size=10000, n_samples=None, res_dir='base'):
    lang_tbar = tqdm(languages, total=len(languages), unit='Language', position=0, leave=True)
    for language in lang_tbar:
        lang_tbar.set_description(f'Current language: {language}')
        license_tbar = tqdm(licenses, total=len(licenses), unit='License', position=1, leave=True)
        for license in license_tbar:
            license_tbar.set_description(f'Current license: {license}')

            print(f"Loading Evaluation Data for {language} with {license} license and {size} samples...")
            eval_data = load_saved_github_code_eval_subset(language, license, size)


            n_samples = len(eval_data) if n_samples is None else n_samples
            ppl = 0
            current_text_len = 0
            results = {'text': [], 'perplexity': []}

            tbar = tqdm(range(n_samples),total=n_samples, unit='Sample', position=0, leave=True, desc=f'Current perplexity: {ppl:.2f}| Current text length: {current_text_len}')
            for i in tbar:
                text = eval_data[i]
                current_text_len = len(text)
                ppl = get_perplexity(text, stride)
                results['text'].append(text[100:])
                results['perplexity'].append(ppl.item())
                tbar.set_description(f'Current perplexity: {ppl:.2f}| Current text length: {current_text_len}')
                tbar.refresh()

            # Save results
            results['avg_perplexity'] = sum(results['perplexity'])/len(results['perplexity'])
            save_results_dict(results, language, license, stride, res_dir)
            print(f"Average perplexity: {results['avg_perplexity']:.2f}")
            print('='*50)


def load_full_model(model_path, tokenizer_path):
    model = AutoModelWithLMHead.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return model, tokenizer


def load_lora_model(model_path):
    config = PeftConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
    model =  AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, use_cache=False)
    model = PeftModel.from_pretrained(model, model_path)
    return model, tokenizer


#### ONE Model

In [ ]:
model_name = "ammarnasr/codegne-finetuned-LoRa-the-stack-java-v3-checkpoint-800"
MODEL, TOKENIZER = load_lora_model(model_name)
MODEL.to(DEVICE)
MODEL.eval()
MAX_LENGTH = MODEL.config.n_positions

In [ ]:
languages = ['Python', 'Java']
licenses = ['mit', 'apache-2.0']
stride = 1024
n_samples = 100
main(languages, licenses, stride, n_samples=n_samples)

#### ALL

In [ ]:
import os
import json
import torch
import pickle
from tqdm.auto import tqdm
from datasets import load_dataset
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelWithLMHead
os.chdir('/content/drive/MyDrive/LLM-for-code-intelligence-Project/LLM-for-code-intelligence/Evaluation/Perplexity')



global DEVICE
global MODEL
global TOKENIZER
global MAX_LENGTH
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_perplexity(text, stride):
    encodings = TOKENIZER(text, return_tensors="pt").to(DEVICE)
    seq_len = encodings.input_ids.size(1)
    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + MAX_LENGTH, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(DEVICE)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = MODEL(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
        nlls.append(neg_log_likelihood)
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl


def save_github_code_eval_subset(language, size=1000):
    """
    Saves a subset of the github code dataset to be used for evaluation.
    :param language: Programming language of the code
    :param size: Size of the subset
    :param shuffle: Whether to shuffle the dataset before taking the subset
    :return: None
    """
    language_lower = language.lower()
    dataset_id = f'ammarnasr/the-stack-{language_lower}-clean'
    ds = load_dataset(dataset_id, split="test", ignore_verifications=True)
    evaluation_dataset = []
    for item in tqdm(ds, total=size, desc="Saving evaluation data Locally"):
        evaluation_dataset.append(item['content'])
    eval_data_name = f"./data/evaluation_data_{language}_{size}.pkl"
    save_dir = os.path.dirname(eval_data_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    with open(eval_data_name, 'wb') as f:
        pickle.dump(evaluation_dataset, f)


def load_saved_github_code_eval_subset(language, size=1000):
    """
    Loads a subset of the github code dataset to be used for evaluation.
    :param language: Programming language of the code
    :param size: Size of the subset
    :return: None
    """
    eval_data_name = f"./data/evaluation_data_{language}_{size}.pkl"
    if not os.path.exists(eval_data_name):
        print(f"Saved evaluation data NOT found. Saving now...")
        save_github_code_eval_subset(language, size)
    else:
        print(f"FOUND saved evaluation data")
    with open(eval_data_name, 'rb') as f:
        evaluation_dataset = pickle.load(f)
    return evaluation_dataset


def save_results_dict(results_dict, language, stride, res_dir):
    """
    Saves a results dictionary to be used for evaluation to a json file.
    :param results_dict: Dictionary containing the results
    :param language: Programming language of the code
    :param size: Size of the subset
    :param res_dir: Directory to save the results
    :return: None
    """
    #check if directory name results exists
    if not os.path.exists('results'):
        os.makedirs('results')
    #check if res_dir exists in results
    final_res_dir = f"./results/{res_dir}"
    if not os.path.exists(final_res_dir):
        os.makedirs(final_res_dir)
    results_name = f"{final_res_dir}/results_{language}_{stride}.json"
    results_name_short = f"{final_res_dir}/results_{language}_{stride}_short.json"

    with open(results_name, 'w') as f:
        json.dump(results_dict, f)
    print(f"Saved results to {results_name}")

    # Delete text to save space
    del results_dict['text']
    with open(results_name_short, 'w') as f:
        json.dump(results_dict, f)
    print(f"Saved short results to {results_name_short}")


def load_full_model(model_path, tokenizer_path):
    model = AutoModelWithLMHead.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return model, tokenizer


def load_lora_model(model_path):
    config = PeftConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
    model =  AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, use_cache=False)
    model = PeftModel.from_pretrained(model, model_path)
    return model, tokenizer


def main(languages, stride, size=10000, res_dir='base'):
    lang_tbar = tqdm(languages, total=len(languages), unit='Language', position=0, leave=True)
    for language in lang_tbar:
        lang_tbar.set_description(f'Current language: {language}')
        eval_data = load_saved_github_code_eval_subset(language, size)
        ppl = 0
        current_text_len = 0
        results = {'text': [], 'perplexity': []}
        tbar = tqdm(range(size),total=size, unit='Sample', position=0, leave=True, desc=f'Current perplexity: {ppl:.2f}| Current text length: {current_text_len}')
        for i in tbar:
            text = eval_data[i]
            current_text_len = len(text)
            ppl = get_perplexity(text, stride)
            results['text'].append(text[100:])
            results['perplexity'].append(ppl.item())
            tbar.set_description(f'Current perplexity: {ppl:.2f}| Current text length: {current_text_len}')
            tbar.refresh()
        results['avg_perplexity'] = sum(results['perplexity'])/len(results['perplexity'])
        save_results_dict(results, language, stride, res_dir)
        print(f"Average perplexity: {results['avg_perplexity']:.2f}")
        print('='*50)




In [ ]:
models_dict = {
    'LoRa-Java': 'ammarnasr/codegen-350M-mono-java',
    'LoRa-Ruby': 'ammarnasr/codegen-350M-mono-ruby',
    'LoRa-Rust': 'ammarnasr/codegen-350M-mono-rust',
    'LoRa-Swift': 'ammarnasr/codegen-350M-mono-swift',
    'Baseline-Mono': 'Salesforce/codegen-350M-mono',
    'Baseline-Multi': 'Salesforce/codegen-350M-multi',

}
languages = ['java', 'ruby', 'rust', 'swift']
stride = 1024
size = 25

In [ ]:
languages_main = ['java', 'ruby', 'rust', 'swift']
for l in languages_main:
  languages = [l]
  for model_short_name, model_long_name in models_dict.items():
      print(f"Current Model: {model_short_name}")

      res_dir = f'{model_short_name}_results_{size}'
      if not os.path.exists('results'):
          os.makedirs('results')
      final_res_dir = f"./results/{res_dir}"
      if not os.path.exists(final_res_dir):
          os.makedirs(final_res_dir)
      results_files = os.listdir(final_res_dir)
      if len(results_files) == 8:
          print(f"Found all 8 results files for {model_short_name}. Skipping...")
          continue
      print(f"Found {len(results_files)} results files for {model_short_name}. Running...")


      if 'LoRa' in model_short_name:
          MODEL, TOKENIZER = load_lora_model(model_long_name)
      else:
          MODEL, TOKENIZER = load_full_model(model_path=model_long_name, tokenizer_path='Salesforce/codegen-350M-mono')
      MODEL.to(DEVICE)
      MODEL.eval()
      MAX_LENGTH = MODEL.config.n_positions
      main(languages, stride, size, res_dir)


Current Model: LoRa-Java
Found 4 results files for LoRa-Java. Running...


  0%|          | 0/1 [00:00<?, ?Language/s]

FOUND saved evaluation data


Current perplexity: 0.00| Current text length: 0:   0%|          | 0/25 [00:00<?, ?Sample/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2521 > 2048). Running this sequence through the model will result in indexing errors


...

  0%|          | 0/1 [00:00<?, ?Language/s]

FOUND saved evaluation data


Current perplexity: 0.00| Current text length: 0:   0%|          | 0/25 [00:00<?, ?Sample/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2388 > 2048). Running this sequence through the model will result in indexing errors


Saved results to ./results/Baseline-Multi_results_25/results_rust_1024.json
Saved short results to ./results/Baseline-Multi_results_25/results_rust_1024_short.json
Average perplexity: 5.76
Current Model: LoRa-Java
Found all 8 results files for LoRa-Java. Skipping...
Current Model: LoRa-Ruby
Found all 8 results files for LoRa-Ruby. Skipping...
Current Model: LoRa-Rust
Found all 8 results files for LoRa-Rust. Skipping...
Current Model: LoRa-Swift
Found all 8 results files for LoRa-Swift. Skipping...
Current Model: Baseline-Mono
Found all 8 results files for Baseline-Mono. Skipping...
Current Model: Baseline-Multi
Found all 8 results files for Baseline-Multi. Skipping...
